# Submodules

In this notebook, you will learn about
- The polynomial submodule: analyzing and building an approximation to existing data
- Random number generation and processing

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd

In notebooks [01_Intro](./01_Intro.ipynb) and [02_How_to_write_efficient_code](./02_How_to_write_efficient_code.ipynb), you looked at how to load the cost of living and quality of life data sets into NumPy and process the data. In this notebook, you will load this data into NumPy again to find the cost of living to get the greatest quality of life. You'll do this in three steps:

1. Merge the "cost of living" and "quality of life" data based upon City
2. Plot the data and look for a trend
3. Create a polynomial fit to estimate the trend and calculate the maximum "quality of life" given "cost of living".

## 1. Merge the `cost of living` and `quality of life` data based upon City

The two data sets have many of the same cities, but not all. First, you load the two data sets into two variable

- `cost_of_living`: the "cost of living" data set
- `quality_of_life`: the "quality of life" data set

In [ ]:
cost_of_living = pd.read_csv('../data/Cost_of_living_index.csv')
quality_of_life = pd.read_csv('../data/quality_of_life_index.csv')

Now, you can `merge` the data sets into one based upon the `'City'`. 

In [ ]:
merged_data = cost_of_living.merge(quality_of_life, how = 'inner', on = ['City'])
merged_data

The merged data matches the City between `cost_of_living` and `quality_of_life` so each row now has both data sets. If a city was missing from either data set it was not included. 

## 2. Plot the data and look for a trend

Now, create 3 arrays with the data of interest:

- `city` is the city where the data was collected e.g. 'Oslo, Norway', 'New York, NY, United States'
- `cost_of_living` is the cost of living plus rent index for each city
- `quality_of_life` is the quality of life index measured for each city

In [ ]:
city = merged_data['City'].values
cost_of_living = merged_data["Cost of Living Plus Rent Index"].values
quality_of_life = merged_data["Quality of Life Index"].values

Now, you can plot the `quality_of_life` vs the `cost_of_living` and label some cities of interest. Lets start off with 3 labels:

- city with highest `cost_of_living`
- city with highest `quality_of_life`
- city with minimum `cost_of_living`

In [ ]:
max_cost_of_living = np.max(cost_of_living)
max_quality_of_life = np.max(quality_of_life)
min_cost_of_living = np.min(cost_of_living)

In [ ]:
plt.plot(cost_of_living, quality_of_life, 'o')
plt.text(max_cost_of_living, quality_of_life[cost_of_living == max_cost_of_living], city[cost_of_living == max_cost_of_living])
plt.text(cost_of_living[quality_of_life == max_quality_of_life], 
         max_quality_of_life, 
         city[quality_of_life == max_quality_of_life])
plt.text(min_cost_of_living, 
         quality_of_life[cost_of_living == min_cost_of_living], 
         city[cost_of_living == min_cost_of_living])
plt.xlabel('cost of living + rent (%)')
plt.ylabel('quality of life')

Above, you have all of the 'quality of life' plotted vs the 'cost of living' for each city. There appears to be a maximum 'quality of life' in the 50-60 \% 'cost of living' index. Next, use the NumPy `polyfit` to create a polynomial to estimate this data set as a quadratic. 

## 3. Create a polynomial fit to estimate the trend and calculate the maximum "quality of life" given "cost of living"

Here, use the function, [`np.polyfit`](https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html) to create a second-order polynomial fit to the 'quality of life' vs 'cost of living' measures. Use the function as such

```python
PP = np.polyfit(x_data, y_data, order)
```

where
- `PP` is the output array that has the polynomial coefficients, highest order first
- `x_data` are the x-axis values: 'cost of living' measurements
- `y_data` are the y-axis values: 'quality of life' measuremnts
- `order` is the polynomial degree: 2 for quadratic e.g. $y(x) = Ax^2 + Bx + C$

In [ ]:
PP = np.polyfit(cost_of_living, quality_of_life, 2)
PP

The output, `PP`, are the polynomial coefficients in an array with shape `(3, )`. One way to evaluate the polynomial at new values is to manually enter the formula as such

```python
x_values = np.linspace(10, 110, 50)
y_poly = PP[0]*x_values**2 + PP[1]*x_values + PP[2]
```

but, there is a better way! You can use the built-in NumPy function, [`np.polyval`](https://numpy.org/doc/stable/reference/generated/numpy.polyval.html) to specify the constants and pass the array of x values. 

```python
x_values = np.linspace(10, 110, 50)
y_poly = np.polyval(PP, x_values)
```

where

- `PP` is an array that has polynomial coefficents ordered from highest order to lowest $y(x) = PP_0 x^n + PP_1 x^{n-1} +...+PP_{n-1} x + PP_n$
- `x_values` is an array that will be evaluated at given `x` locations it can be a floating point number or a 1D array
- `y_poly` is the returned array evaluated at each `x_values` location given coefficients in `PP`

Here, define the `cost_values` to go from 10\% to 110\% with 50 steps and evaluate `quality_poly` at each cost. 

In [ ]:
cost_values = np.linspace(10, 110, 50)
quality_poly = np.polyval(PP, cost_values)

The `quality_poly` is now a quadratic best-fit data set for the 'quality of life' given a 'cost of living' index. Plot the result to look for the maximum of the polynomial. 

In [ ]:
plt.plot(cost_of_living, quality_of_life, 'o')
plt.plot(cost_values, quality_poly)
plt.xlabel('cost of living + rent (%)')
plt.ylabel('quality of life')

This looks like a nice fit for the data, but __Where should I live?__

You will find some of the cities that are close to the optimium "cost of living". You will print out the cities as an array, then add some annotations to the plot you created above. Approach this in 3 steps:

1. find the optimum cost using `np.max()`
2. find all the "cost of living" values within 5\% of this optimum
3. print the cities and display four on the "quality of life" vs "cost of living" graph

First, assign the variable `optimum_cost` to the cost that results in the maximum polynomial result. 

In [ ]:
optimum_cost = cost_values[quality_poly == quality_poly.max()]

The `quality_poly == quality_poly.max()` creates an array of `True`/`False` statements that are passed into `cost_values`. There is only one maximum value in `quality_poly` so `optimum cost` is an array with one value, shape is `(1, )`. 

Second, use the [`np.isclose`](https://numpy.org/doc/stable/reference/generated/numpy.isclose.html) to assign `optimum_bools` to an array of `True`/`False` statements to find
- `optimum_cities` the city names closest to the optimum quality of life
- `optimum_costs` the "cost of living" values closest to the optimum quality of life
- `optimum_quality` the "quality of life" values closest to the optimum quality of life

In [ ]:
optimum_bools = np.isclose(cost_of_living, optimum_cost, atol=5)

optimum_cities = city[optimum_bools]
optimum_costs = cost_of_living[optimum_bools]
optimum_quality = quality_of_life[optimum_bools]

Third, you want to see which cities optimize "cost of living" to "quality of life". The list of cities in `optimum_cities` are the cities closest to the quadratic fit maximum "quality of life" measurements. Take a look at the list of names, "cost of living", and "quality of life" measures. 

> __Note__: Here you use a combination of `print` and string [`format`](https://docs.python.org/3.4/library/functions.html#format) work to get the data into a table. The `{:30s}` prints out a string that is always 30 characters long. This ensures the next column, "cost of living" is in the right location after the tab, `\t`. 



In [ ]:
print('City:\t\t\tCost of living:\tQuality of life:')
for i in range(len(optimum_cities)):
    print('{:30s} \t {} \t {}'.format(optimum_cities[i], optimum_costs[i], optimum_quality[i]))

You can print out all of these city names on the plot, but it gets messy. Instead, use the `plt.text` to show every fourth city name (or 3 cities). Create the plot with the data and quadratic fit then use a `for`-loop to add the text for the cities. 

In [ ]:
plt.plot(cost_of_living, quality_of_life, 'o')
plt.plot(cost_values, quality_poly)
for i in range(0, len(optimum_costs), 4):
    plt.text(optimum_costs[i], optimum_quality[i], optimum_cities[i])
plt.xlabel('cost of living + rent (%)')
plt.ylabel('quality of life')

So, if you're in the US it looks like Washington, DC (East coast) and Seattle, WA (West coast) are good options or in Europe Oslo, Norway is an optimal place to live too. 

## 4. Saving data for later use with `np.savez`

Now, let's say after all the hard work we want to save your arrays for later use. We can save these arrays to disk using the NumPy `.savez()` function.

In [ ]:
np.savez?

The `.npz` file format is the most appropriate and efficient to use with NumPy arrays, so we will use it here. Choosing a filename of `optimum.npz`, we can save the array by doing

In [ ]:
np.savez('optimum.npz', optimum_costs, optimum_quality)

By listing the contents of the current folder, we can see that the file we just created is indeed there:

In [ ]:
!ls

To recover these arrays, we can use

In [ ]:
npzfile = np.load('optimum.npz')

The `npzfile` object now contains both arrays that were stored in this file.  

In [ ]:
npzfile.files

In [ ]:
npzfile['arr_0']

Let's test that we have actually recovered the correct values for all elements in our `optimum_costs` array:

In [ ]:
np.equal(npzfile['arr_0'], optimum_costs) 

---

### Self-assessment 1

---

## Read more
- [NumPy functions and methods overview](https://numpy.org/devdocs/user/quickstart.html#functions-and-methods-overview)
- [NumPy Quickstart guide](https://numpy.org/devdocs/user/quickstart.html)
- [NumPy for absolute beginners](https://numpy.org/devdocs/user/absolute_beginners.html)